In [ ]:
# !pip install pyspark


In [ ]:
import networkx as nx

import pyspark
from pyspark import SparkContext
import collections

In [ ]:
sc = SparkContext.getOrCreate()

In [ ]:
data = {}
# Đọc dữ liệu từ file data
a_file = open("data.txt")
for line in a_file:
  key, value = line.split()
  data[int(key)] = list(map(int, value.split(',')))


# Spark

In [ ]:
# Lấy các Page trong data và mặc định các Page có rank là 1
ranks = sc.parallelize(data.keys()).map(lambda x : (x, 1.))
print(ranks.take(3))

[(1, 1.0), (2, 1.0), (3, 1.0)]


In [ ]:
# Lấy các Page và các Page link với Page đó
links = sc.parallelize(data.items())

In [ ]:
# Hàm này sẽ tính độ liên kết của các Page với rank hiện tại của Page đang xét
def computeContribs(node_urls_rank):
    # lấy các page liên kết và rank của page đang xét
    _, (urls, rank) = node_urls_rank
    # Tính số lượng page liên kết đến page đang xét
    neighbors_urls = len(urls)
    # Tính rank cho mỗi page liên kết dựa trên rank hiện tại của page đang xét
    for url in urls:
        yield url, rank / neighbors_urls

In [ ]:
from operator import add
from time import time

start = time()
for iteration in range(10):
    # Tính độ liên kết của từng dòng dữ liệu
    contribs = links.join(ranks).flatMap(computeContribs)
    # Tính tổng rank của từng Page sau khi tính độ liên kết
    ranks = contribs.reduceByKey(add)
    # Chuẩn hóa rank của từng page bằng công thứ rank*d + (1-d)
    # Tham số d mô phỏng  xác suất của một người dùng ngẫu nhiên 
    # liên tục nhấp vào liên kết trên trang khi họ truy cập vào Website.
    # d ở đây sẽ được lấy giá trị là 0.85
    ranks = ranks.mapValues(lambda rank: rank * 0.85 + 0.15)
spark_time = time() - start

In [ ]:
Ranking = ranks.sortBy(lambda x: x[1],ascending=False)
temp = Ranking.take(10)
print("--- Spark ---")
for a in temp:
    print(a)

--- Spark ---
(8019, 2.0351773744622372)
(585, 1.9868996300502182)
(8163, 1.913896934148332)
(9345, 1.9087122291949876)
(7764, 1.861835617781206)
(3596, 1.84113217608709)
(8457, 1.8391313952159771)
(3011, 1.838931471156723)
(2694, 1.8367522943328456)
(8142, 1.805481425644822)


# NetworkX

In [ ]:
link_graph = nx.DiGraph(data)
start = time()
pr=nx.pagerank(link_graph,0.85)
networkx_time = time()-start
pr = dict(sorted(pr.items(), key=lambda item: item[1], reverse=True))

print("--- NetworkX ---")
# Print top 10 Page
i=0
for a in pr.items():
    if i == 10:
      break
    print(a)
    i+=1


--- NetworkX ---
(8019, 0.00020502640114221043)
(585, 0.0001990999099256514)
(9345, 0.00019067356916954825)
(8163, 0.0001903096388005121)
(7764, 0.00018547773842801052)
(3011, 0.00018474211418624688)
(3596, 0.000183874666001705)
(8457, 0.00018329631038849252)
(2694, 0.00018258415535935915)
(8142, 0.00018094893867960594)


# Non Spark

In [ ]:
keys = list(data.keys())
ranks = dict()
for a in keys:
    ranks[a] = 1.

start = time()
for iteration in range(10):
    new_ranks = dict()
    for key in ranks.keys():
        if key in data:
            for item in data[key]:
                if item in new_ranks:
                    new_ranks[item] += ranks[key]/len(data[key])
                else:
                    new_ranks[item] = ranks[key]/len(data[key])
    for key in new_ranks.keys():
        new_ranks[key] = new_ranks[key] * 0.85 + 0.15
    ranks = new_ranks
non_spark_time = time() - start
new_ranks = {k: v for k, v in sorted(new_ranks.items(), key=lambda item: item[1], reverse=True)}
i=0
print("--- Non Spark ---")

for a in new_ranks.items():
  if i == 10:
    break
  print(a)
  i+=1

--- Non Spark ---
(8019, 2.0351773744622372)
(585, 1.9868996300502153)
(8163, 1.913896934148331)
(9345, 1.9087122291949867)
(7764, 1.8618356177812077)
(3596, 1.8411321760870898)
(8457, 1.839131395215977)
(3011, 1.8389314711567226)
(2694, 1.8367522943328471)
(8142, 1.805481425644823)


# Time

In [ ]:
print("Spark: ", spark_time)
print("NetWorkX: ", networkx_time)
print("Non-Spark: ", non_spark_time)

Spark:  0.7023742198944092
NetWorkX:  30.213271141052246
Non-Spark:  32.16502523422241
